In [2]:
import pandas as pd
import numpy as np

In [35]:
title_basics_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\TV_Movie_sampled.tsv", sep="\t")
title_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120134 entries, 0 to 120133
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          120134 non-null  object
 1   titleType       120134 non-null  object
 2   primaryTitle    120134 non-null  object
 3   originalTitle   120134 non-null  object
 4   isAdult         120134 non-null  int64 
 5   startYear       120134 non-null  object
 6   endYear         120134 non-null  object
 7   runtimeMinutes  120134 non-null  object
 8   genres          120134 non-null  object
dtypes: int64(1), object(8)
memory usage: 8.2+ MB


In [4]:
base_ele = ["tconst", "titleType", "primaryTitle", "originalTitle", "isAdult", "startYear", "endYear", "runtimeMinutes"]
TV_Movie = title_basics_df[base_ele]

In [5]:
# Now drop tvEpisode entries from TV_Movie
TV_Movie = TV_Movie[TV_Movie["titleType"] != "tvEpisode"]


In [6]:
# replace "," or ";" to "/" in the string
TV_Movie['primaryTitle'] = TV_Movie['primaryTitle'].str.replace(r',','/', regex=False)
TV_Movie['primaryTitle'] = TV_Movie['primaryTitle'].str.replace(r';','/', regex=False)
TV_Movie['originalTitle'] = TV_Movie['originalTitle'].str.replace(r',','/', regex=False)
TV_Movie['originalTitle'] = TV_Movie['originalTitle'].str.replace(r';','/', regex=False)

In [7]:
# replace \N to ''
TV_Movie['tconst'] = TV_Movie['tconst'].replace(r'\N','')
TV_Movie['titleType'] = TV_Movie['titleType'].replace(r'\N','')
TV_Movie['primaryTitle'] = TV_Movie['primaryTitle'].replace(r'\N','')
TV_Movie['startYear'] = TV_Movie['startYear'].replace(r'\N','')
TV_Movie['endYear'] = TV_Movie['endYear'].replace(r'\N','')
TV_Movie['runtimeMinutes'] = TV_Movie['runtimeMinutes'].replace(r'\N','')

In [8]:
# delete data rows where "tconst" or "primaryTitle" is None based on the constraint.
TV_Movie[["tconst", "primaryTitle"]].dropna()
TV_Movie = TV_Movie[TV_Movie["primaryTitle"].isna()==False]

In [9]:
import psycopg2
import io

In [10]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres.rnxnnqsyjcircloxrryq",
    password="cse560courseproject",
    host="aws-0-us-east-1.pooler.supabase.com",
    port=6543
)
cur = conn.cursor()

## TV Movie Relation

In [12]:


# for tv_movie relation
buffer = io.StringIO()
TV_Movie.to_csv(buffer, index=False, header=False)
buffer.seek(0)


0

In [13]:
conn.rollback()

In [14]:

# upload data to tv_movie
cur.copy_from(buffer, 'tv_movie', sep=',', null='')
conn.commit()

## Genres

In [36]:
title_basics_df['genres'] = title_basics_df['genres'].str.split(',')
genres_explode = title_basics_df.explode('genres')

In [46]:
genres_set = set()
for (i, row) in title_basics_df.iterrows():
    for g in row['genres']:
        genres_set.add(g)
genres_set.remove('\\N')

In [49]:
genres_df = pd.DataFrame(list(genres_set),columns=['genre'])

In [51]:
# for tv_movie relation
buffer = io.StringIO()
genres_df.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [52]:
cur.copy_from(buffer, 'genres', sep=',', null='', columns=('genre',))
conn.commit()

## Genre Movie

In [54]:
genres_df = genres_df.reset_index().rename(columns={'index':'genreID'})

In [56]:
genres_df['genreID'] = genres_df['genreID'] + 1

In [60]:
genres_explode['genre'] = genres_explode['genres']

In [66]:
genre_movie = genres_explode.merge(genres_df,on='genre',how='inner')

In [67]:
genre_movie = genre_movie[['genreID','tconst']]

In [69]:
tconst_set = set(TV_Movie['tconst'])

In [70]:
genre_movie = genre_movie[genre_movie['tconst'].isin(tconst_set)]

In [71]:
# for tv_movie relation
buffer = io.StringIO()
genre_movie.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [72]:
cur.copy_from(buffer, 'genres_movie', sep=',', null='')
conn.commit()